In [158]:
import os
import re
import json
from collections import namedtuple

from yargy import Parser
from yargy import rule
from yargy import predicates
from yargy import or_, and_, not_
from yargy import pipelines
from yargy.interpretation import fact, attribute
from ipymarkup import show_markup
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
df = pd.read_excel("data/data.xlsx", engine="openpyxl")
df.columns = ["desc", "code"]
fesi = df[df.code == 7202210000]

In [3]:
vals = fesi.desc.values
vals = vals.tolist()

In [4]:
vals[:5]

['ФЕРРОСИЛИЦИЙ ФС70(SI-72% MIN) ФР.10-50 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-604,686 БАЗОВЫХ ТОНН ВЕС НЕТТО-572000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ',
 'ФЕРРОСИЛИЦИЙ ФС70(SI-70% MIN) ФР.10-50,10-100,40-100 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-114,171 БАЗОВЫХ ТОНН ВЕС НЕТТО-108000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ',
 'ФЕРРОСИЛИЦИЙ ФС75(SI-75% MIN) ФР.10-50, 40-80, 50-100 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-616,000 БАЗОВЫХ ТОНН ВЕС НЕТТО-600000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ',
 'ФЕРРОСИЛИЦИЙ ФС75(SI-72% MIN) ФР.0-5 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-135,520 БАЗОВЫХ ТОНН ВЕС НЕТТО-132000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ',
 'ФЕРРОСИЛИЦИЙ ФС65 (СОДЕРЖАНИЕ SI 55-65%, С-1,5%) ИЗГОТОВЛЕНО ПО ГОСТ 1415-93, РАЗМЕР 10-200 ММ., (МЕНЕЕ 10 ММ / БОЛЕЕ 200ММ - 10%).']

In [79]:
def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        spans = [_.span for _ in matches]
        show_markup(line, spans)
        
def join_spans(text, spans):
    spans = sorted(spans)
    return ' '.join(
        text[start:stop]
        for start, stop in spans
    )
        
class Match(object):
    def __init__(self, fact, spans):
        self.fact = fact
        self.spans = spans
        

class Extractor(object):
    """This class wraps up an 'or_'-based parser to create a single object."""
    def __init__(self, union_rule_obj, wrapper_obj):
        self.union_rule_obj_parser = Parser(union_rule_obj)
        self.wrapper_parser = Parser(wrapper_obj)

    def __call__(self, text):
        matches = self.union_rule_obj_parser.findall(text)
        spans = [_.span for _ in matches]

        line = join_spans(text, spans)
        matches = list(self.wrapper_parser.findall(line))
        fact = None
        if matches:
            match = matches[0]
            fact = match.fact

        return Match(fact, spans)

In [58]:
INT = rule(predicates.type('INT'))

FLOAT = rule(
    INT,
    predicates.in_({',', '.'}),
    predicates.in_('1234567890').repeatable()  # не больше одного знака после запятой
)

INT_OR_FLOAT = rule(or_(INT, FLOAT))
PCT = rule(
    INT_OR_FLOAT, '%'
)
INT_FLOAT_PCT = rule(or_(INT_OR_FLOAT, PCT))

SLASH = predicates.eq('/')
DASH = predicates.eq('-')
SEMICOLON = predicates.eq(':')
DASH_OR_SLASH = rule(or_(DASH, SLASH))
DOT = predicates.eq('.')
COMMA = predicates.eq(',')

In [93]:
fesi_fact = fact("fesi", ["min", "max"])
frac_fact = fact("frac", [attribute("mm").repeatable()])

total_fesi_fact = fact("fesi", ["min_si", "max_si", attribute("mm").repeatable()])

In [127]:
class FeSi(total_fesi_fact):
    @property
    def normalized(self):
        self.min_si = self.min_si.replace('%', '')
        self.min_si = self.min_si.strip()
        self.min_si = self.min_si.replace(' ', '.')
        self.min_si = self.min_si.replace(',', '.')
        self.min_si = self.min_si.replace('..', '.') 
        self.min_si = float(self.min_si)
        if self.max_si is not None:
            self.max_si = self.max_si.replace('%', '')
            self.max_si = self.max_si.strip()
            self.max_si = self.max_si.replace(',', '.')
            self.max_si = self.max_si.replace(' ', '.')
            self.max_si = self.max_si.replace('..', '.') 
            self.max_si = float(self.max_si)
        ranges = []
        for m in self.mm:
            rs = m.split(", ")
            rs = [r.strip() for r in rs if r != ""]
            rs = [r.replace(",", ".") for r in rs]
            for r in rs:
                str_ints = r.split("-")
                try:
                    ranges.append({"frac_min":float(str_ints[0]), "frac_max":float(str_ints[1])})
                except ValueError:
                    ranges.append({"frac_min":str_ints[0], "frac_max":str_ints[1]})
        return {"si_min": self.min_si, "si_max": self.max_si, "frac": ranges}

In [8]:
# FeSI_ = namedtuple('FeSi', ['min', 'max'])

# class FeSi(fesi_fact):
#     @property
#     def normalized(self):
#         self.min = self.min.replace('%', '')
#         self.min = self.min.strip()
#         self.min = self.min.replace(' ', '.')
#         self.min = self.min.replace(',', '.')
#         self.min = self.min.replace('..', '.') 
#         self.min = float(self.min)
#         if self.max is not None:
#             self.max = self.max.replace('%', '')
#             self.max = self.max.strip()
#             self.max = self.max.replace(',', '.')
#             self.max = self.max.replace(' ', '.')
#             self.max = self.max.replace('..', '.') 
#             self.max = float(self.max)
#         return FeSI_(self.min, self.max)
        

In [68]:
# Frac_ = namedtuple("Frac", ["min", "max"])

# class Frac(frac_fact):
#     @property
#     def normalized(self):
#         ranges = []
#         for m in self.mm:
#             rs = m.split(", ")
#             rs = [r.strip() for r in rs if r != ""]
#             rs = [r.replace(",", ".") for r in rs]
#             for r in rs:
#                 str_ints = r.split("-")
#                 fr = Frac_(str_ints[0], str_ints[1])
#                 ranges.append(fr)
#         return ranges
            

In [128]:
SI = pipelines.morph_pipeline(
    ['SI', 'КРЕМНИЙ']
)

FESI_RANGE = rule(
              INT_FLOAT_PCT.interpretation(FeSi.min_si),
              DASH_OR_SLASH.optional(),
              INT_FLOAT_PCT.interpretation(FeSi.max_si).optional()
).interpretation(FeSi)

In [129]:
FESI_RULE = rule(or_(
         rule(SI,
              DASH.optional(),
              FESI_RANGE
              ),
        rule(predicates.normalized('СОДЕРЖАЩИЙ'),
             predicates.normalized("БОЛЕЕ").optional(),
             FESI_RANGE,
             SI.optional()
            ),
        rule(predicates.normalized('СОДЕРЖАНИЕ'),
             SI.optional(),
             predicates.normalized("НЕ МЕНЕЕ").optional(),
             FESI_RANGE
            ),
         )).interpretation(FeSi)

In [12]:
# parser = Parser(FESI_RULE)
# empty=0
# f = []
# for line in vals:
#     match = parser.findall(line)
#     try:
#         matches = [x.fact.normalized for x in match]
#         if len(matches) == 1:
#             f.append(matches[0])
#         elif len(matches) > 1:
#             f.append(matches[-1])
# #         print(matches)
#     except ValueError as err:
#         empty+=1
# #         print(line)
#         print(err)
#         continue
#     if len(matches) == 0:
# #         print(line)
#         empty+=1
# print(f"Found {(len(vals)-empty)/len(vals):.2%} matches")

Found 97.52% matches


In [13]:
len(set(f))

223

In [14]:
vals[:5]

['ФЕРРОСИЛИЦИЙ ФС70(SI-72% MIN) ФР.10-50 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-604,686 БАЗОВЫХ ТОНН ВЕС НЕТТО-572000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ',
 'ФЕРРОСИЛИЦИЙ ФС70(SI-70% MIN) ФР.10-50,10-100,40-100 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-114,171 БАЗОВЫХ ТОНН ВЕС НЕТТО-108000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ',
 'ФЕРРОСИЛИЦИЙ ФС75(SI-75% MIN) ФР.10-50, 40-80, 50-100 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-616,000 БАЗОВЫХ ТОНН ВЕС НЕТТО-600000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ',
 'ФЕРРОСИЛИЦИЙ ФС75(SI-72% MIN) ФР.0-5 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-135,520 БАЗОВЫХ ТОНН ВЕС НЕТТО-132000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ',
 'ФЕРРОСИЛИЦИЙ ФС65 (СОДЕРЖАНИЕ SI 55-65%, С-1,5%) ИЗГОТОВЛЕНО ПО ГОСТ 1415-93, РАЗМЕР 10-200 ММ., (МЕНЕЕ 10 ММ / БОЛЕЕ 200ММ - 10%).']

In [130]:
FR = pipelines.morph_pipeline(
    ["ФР.", "ФР", "РАЗМЕР", "ФР:.", "ФР.:", "ФР:", "ФРАКЦИЯ"]
)

FR_RANGE = rule(
    INT_OR_FLOAT,
    DASH,
    INT_OR_FLOAT,
    COMMA.optional()
).repeatable().interpretation(FeSi.mm)

FR_RULE = rule(
    FR,
    FR_RANGE.repeatable(),
    predicates.eq("ММ").optional()
).interpretation(FeSi)

In [131]:
# parser = Parser(FR_RULE)
# for line in vals[1000:1500]:
#     matches = parser.findall(line)
# #     spans = [_.span for _ in matches]
# #     show_markup(line, spans)
#     try:
#         facts = [_.fact.normalized for _ in matches]
#     except IndexError as err:
#         print("INDEXERROR: ", line)
#         continue
#     if len(facts)==0:
# #         print("FAILED: ", line)
#           continue
# #     print(facts)

In [50]:
parser = Parser(FR_RULE)
line = """
ФЕРРОСИЛИЦИЙ ФС70(SI-70% MIN) ФР.10-50, 10-100, 50-100 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-114,171 БАЗОВЫХ ТОНН ВЕС НЕТТО-108000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ
"""
matches = parser.findall(line)
# spans = [_.span for _ in matches]
facts = [_.fact.normalized for _ in matches]
# show_markup(line, spans)
print(facts)

[[Frac(min='10', max='50'), Frac(min='10', max='100'), Frac(min='50', max='100')]]


In [133]:
line = """
ФЕРРОСИЛИЦИЙ ФС70(SI-70% MIN) ФР.10-50, 10-100, 50-100 ММ ГОСТ 1415-93 БАЗОВЫЙ ВЕС-114,171 БАЗОВЫХ ТОНН ВЕС НЕТТО-108000 КГ ИСПОЛЬЗУЮТСЯ ДЕРЕВЯННЫЕ РЕКВИЗИТЫ КРЕПЛЕНИЯ
"""
AD = or_(FESI_RULE, FR_RULE).interpretation(FeSi)
WRAPPER = rule(FESI_RULE, FR_RULE.optional()).interpretation(FeSi)
# match = extractor(line)
# wrapper_parser = Parser(WRAPPER)
# ad_parser = Parser(AD)
# matches = list(wrapper_parser.findall(line))
extractor = Extractor(AD, WRAPPER)
match = extractor(line)
pd.DataFrame(match.fact.normalized)

,si_min,si_max,frac
0,70.0,None,"{'frac_min': 10.0, 'frac_max': 50.0}"
1,70.0,None,"{'frac_min': 10.0, 'frac_max': 100.0}"
2,70.0,None,"{'frac_min': 50.0, 'frac_max': 100.0}"


In [137]:
fesi.head()

,desc,code
12,ФЕРРОСИЛИЦИЙ ФС70(SI-72% MIN) ФР.10-50 ММ ГОСТ...,7202210000
24,"ФЕРРОСИЛИЦИЙ ФС70(SI-70% MIN) ФР.10-50,10-100,...",7202210000
92,"ФЕРРОСИЛИЦИЙ ФС75(SI-75% MIN) ФР.10-50, 40-80,...",7202210000
344,ФЕРРОСИЛИЦИЙ ФС75(SI-72% MIN) ФР.0-5 ММ ГОСТ 1...,7202210000
348,"ФЕРРОСИЛИЦИЙ ФС65 (СОДЕРЖАНИЕ SI 55-65%, С-1,5...",7202210000


In [140]:
codes = pd.read_excel("data/data_1.xlsx", engine="openpyxl", sheet_name="codes")

In [143]:
codes.columns = ["code", "name", "_"]

In [152]:
fesi = fesi.merge(codes, left_on="code", right_on="code", how="left")

In [160]:
AD = or_(FESI_RULE, FR_RULE).interpretation(FeSi)
WRAPPER = rule(FESI_RULE, FR_RULE.optional()).interpretation(FeSi)
extractor = Extractor(AD, WRAPPER)


In [165]:
matches = []
for idx, r in tqdm(fesi.iterrows()):
    match = extractor(r.values[0])
    try:
        matches.append({
            "code": r.values[1],
            "name": r.values[2],
            **match.fact.normalized
        })
    except AttributeError:
        matches.append({
            "code": r.values[1],
            "name": r.values[2],
        })


0it [00:00, ?it/s]

[{'code': 7202210000, 'name': 'Ферросилиций, содержащий более 55% кремния', 'si_min': 72.0, 'si_max': None, 'frac': [{'frac_min': 10.0, 'frac_max': 50.0}]}, {'code': 7202210000, 'name': 'Ферросилиций, содержащий более 55% кремния', 'si_min': 70.0, 'si_max': None, 'frac': [{'frac_min': 10.0, 'frac_max': 50.1}, {'frac_min': 40.0, 'frac_max': 100.0}]}, {'code': 7202210000, 'name': 'Ферросилиций, содержащий более 55% кремния', 'si_min': 75.0, 'si_max': None, 'frac': [{'frac_min': 10.0, 'frac_max': 50.0}, {'frac_min': 40.0, 'frac_max': 80.0}, {'frac_min': 50.0, 'frac_max': 100.0}]}, {'code': 7202210000, 'name': 'Ферросилиций, содержащий более 55% кремния', 'si_min': 72.0, 'si_max': None, 'frac': [{'frac_min': 0.0, 'frac_max': 5.0}]}, {'code': 7202210000, 'name': 'Ферросилиций, содержащий более 55% кремния', 'si_min': 55.0, 'si_max': 65.0, 'frac': [{'frac_min': 10.0, 'frac_max': 200.0}]}, {'code': 7202210000, 'name': 'Ферросилиций, содержащий более 55% кремния', 'si_min': 75.0, 'si_max': Non

In [168]:
res = pd.DataFrame(matches)

In [175]:
res[["frac-1", "frac-2", "frac-3", "frac-4", "frac-5"]] = res.frac.apply(pd.Series)

In [177]:
res.drop("frac", axis=1, inplace=True)

In [180]:
res.to_excel("data/fesi_exmpl.xlsx", index=False)